47316 Advanced Computational Tools for Energy Materials

# Activation Energy for H$_2$ formation

In this exercise, we want to calculate the activation energy for H$_2$ formation on close-packed metal surfaces using the RPBE exchange-correlation functional.

## Questions
1. Calculate the bulk lattice constant(s) with RPBE. How do they compare to experimental lattice constants? Include computational details (plane-wave cutoff, k-point sampling).

2. Set up a close packed surface using ASE. You can use the built-in functions (<https://wiki.fysik.dtu.dk/ase/ase/build/surface.html>). See surface.py script below for inspiration on how to do this. For this problem, it is appropriate to use at least a 2x2 surface cell with 3 close packed layers (two bottom layers fixed).

3. Calculate the dissociate H adsorption energy at different high-symmetry sites, and identify the most stable adsorption site. Here we define the dissociative adsorption energy as $$E_\text{ads} = E(H^*) - E(*) - \frac{1}{2}E(H_2(g))$$ where $E(H^*)$ is the DFT energy of the slab with an H adsorbate, $E(*)$ is the DFT energy of the pristine slab, and $E(H_2(g))$ is the DFT energy of the $H_2(g)$ molecule. Report the total energies and show images of the optimized $H*$ structures.
$$E_\text{DFT}(H_2) = ?$$
$$E_\text{DFT}(*) = ?$$


| **Site**     | $E_\text{DFT}$(H*)               | $E_\text{ads}$               |
| ------------ | -------------------------------- | ---------------------------- |
| Atop         |                                  |                              |
| Bridge       |                                  |                              |
| HCP          |                                  |                              |
| FCC          |                                  |                              |


4. Create and optimize initial and final states for the Tafel reaction: $2 H^* \rightarrow H_2 + 2*$. What is the reaction energy?

5. Due to the small size of the surface cell, the initial state has 0.5 ML $H^*$ coverage. What is the (average) dissociative adsorption energy at 0.5 ML $H^*$ coverage?

6. Calculate the activation energy for the Tafel reaction using the CI-NEB method. As a rule of thumb, images should be separated by $0.5-1$ Å. Show images of the initial, final and transition states and energy diagram along the minimum energy pathway (MEP). Does the path look reasonable? If not, it might be beneficial to modify the initial path by hand or try a different initial state.

7. Perform a vibrational analysis of the transition state and initial state. is the identified transition state really a saddle point? Plot the rate constant for the Tafel reaction as function of temperature in the interval from 180 K to 680 K. At which conditions would you expect the metal to be a good catalyst for H$_2$ production?

#### Below we have provided you with some scripts you can use for inspiration

It is recommended to keep your simulations in separated folders. If writing to a different folder, than where the script is being executed, remember to create that folder before running the simulation.

Should you run into the issue of "Broken Symmetry" with the GPAW calculator, try adding the option
```python
    symmetry={'point_group': False}
```
to your calculator object. This will significantly increase the time it takes to run, so keep that in mind.

Remember to add enough processors to the NEB, and that the number of processors you use should be divisible by the number of images.

In [ ]:
%%writefile surface.py
from ase.build import fcc111, add_adsorbate
from ase.optimize import BFGS
from ase.constraints import FixAtoms

from gpaw import GPAW, FermiDirac, PW

# a - lattice parameter from bulk optimization
# Modify this according to your assignment
atoms = fcc111('Cu', size=(2, 2, 3), vacuum=7.5, a=3.679)
#add_adsorbate(atoms, 'H', 1.3, position='fcc', offset=(0,0))  # Initial state for NEB
#add_adsorbate(atoms, 'H', 1.3, position='fcc', offset=(1,0))  # Final state for NEB

# Plane-Wave mode requires periodic boundary conditions in all directions
atoms.set_pbc(True)  

mask=[atom.tag > 1 for atom in atoms]
atoms.set_constraint(FixAtoms(mask=mask))
#atoms.rattle() # escape high symmetry saddle points?

if 0: # for viewing only
    from ase.visualize import view
    view(atoms)
    assert False                  # Stop the script

calc = GPAW(occupations=FermiDirac(0.10),
            xc='RPBE',
            kpts=(4, 4, 1),
            maxiter=600,
            mode=PW(400.),
            spinpol=False,  # Spin polarized?
            txt='surface/Cu111.out',
            convergence={'energy': 0.0005,
                         'density': 1.0e-4,
                         'eigenstates': 4.0e-8,
                         'bands': 'occupied'})

atoms.set_calculator(calc)
dyn = BFGS(atoms, trajectory='surface/qn.traj', logfile='surface/qn.log')
dyn.run(fmax=0.05)

In [ ]:
!qsub -t 1 -p 8 surface.py

In [ ]:
%%writefile neb.py
from ase.io import read, Trajectory
from ase.constraints import FixAtoms
from ase.neb import NEB
from ase.optimize import BFGS
from ase.parallel import rank, size

from gpaw import GPAW, FermiDirac, PW

Nimages = 8  # Number of images
n = size // Nimages # number of cpu's per image
j = 1 + rank // n  # my image number
assert Nimages * n == size

initial = read('initial_for_NEB/qn.traj@-1') # path to the initial state
final = read('final_for_NEB/qn.traj@-1') # path to the final state
# Refer to the last structure in any trajectory file xyz.traj by xyz.traj@-1

constraint = FixAtoms(mask=[atom.tag > 1 for atom in initial]) # modify as needed

images = [initial]

for i in range(Nimages):
    ranks = range(i * n, (i + 1) * n)
    image = initial.copy()
    if rank in ranks:
        calc = GPAW(xc='RPBE',
                    mode=PW(400.),
                    kpts=(4, 4, 1),
                    spinpol=False,  # Spin polarized?
                    occupations=FermiDirac(0.10),
                    convergence={'energy': 0.0005,
                                 'density': 1.0e-4,
                                 'eigenstates': 4.0e-8,
                                 'bands': 'occupied'},
                    txt='NEB/neb{:d}.txt'.format(j),
                    communicator=ranks)

        image.set_calculator(calc)

    image.set_constraint(constraint)
    images.append(image)

images.append(final)

In [ ]:
!qsub -t 8 -p 16 neb.py

In [ ]:
!ase gui -n -1 NEB/ci-neb?.traj  # View CI-NEB.

In [ ]:
%%writefile vib.py
from ase.io import read
from ase.vibrations import Vibrations

from gpaw import GPAW, FermiDirac, PW

slab = read('NEB/ci-neb2.traj@-1') # read initial or transition state structure

calc = GPAW(occupations=FermiDirac(0.10),
            xc='RPBE',
            kpts=(4, 4, 1),
            mode=PW(400.),
            maxiter=600,
            spinpol=False,  # Spin polarized?
            symmetry='off',
            txt='vib/Cu111.out',
            convergence={'energy': 0.0005,
                         'density': 1.0e-4,
                         'eigenstates': 4.0e-8,
                         'bands': 'occupied'})

slab.set_calculator(calc)

# Which modes are needed?
vib = Vibrations(slab, name='vib/Cu111_vib', indices=(12, 13,), nfree=2, delta=0.01)
vib.run()
vib.summary()

for i in range(6):
    vib.write_mode(i) # write modes

In [ ]:
!qsub -t 1 -p 8 vib.py